<a href="https://colab.research.google.com/github/Bhabuk10/Fake_news_prediction/blob/main/Fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing Dependencies


In [1]:
import numpy as np
import pandas as pd 
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data preprocessing


In [4]:
news= pd.read_csv("/content/drive/MyDrive/news.csv")

In [5]:
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
news.shape

(20800, 5)

In [7]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [8]:
# counting number of missing values in dataset

news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
# replacing null values with empty/null strings 

news= news.fillna('')

In [10]:
# merging the author and title column

news['content']= news['author']  + news['title']
print(news['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [13]:
# separating data(content column ) and label 

X = news.drop(columns='label', axis=1)
Y= news['label']


In [15]:
X

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.comWhy the Truth Might Get You ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss15 Civilians Killed In Single U...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard PortnoyIranian woman jailed for fiction...
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel AbramsMacy’...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex AnsaryNATO, Russia To Hold Parallel Exerc..."


In [25]:
# stemming 

port_stem = PorterStemmer()

In [26]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z] ',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content



In [27]:
news['content']= news['content'].apply(stemming)

In [28]:
print(news['content'])

0        darrel lucushous dem aid didn’t even see comey...
1        daniel j flynnflynn hillari clinton big woman ...
2               consortiumnews.comwhi truth might get fire
3        jessica purkiss1 civilian kill singl us airstr...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp t.i. trump ’poster child whit...
20796    benjamin hoffmann.f.l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmacy’ said r...
20798    alex ansarynato russia hold parallel exercis b...
20799                      david swansonwhat keep f-3 aliv
Name: content, Length: 20800, dtype: object


In [29]:
# separating data and label 

X = news['content'].values
Y = news['label'].values

In [31]:
Y

array([1, 0, 1, ..., 0, 1, 1])

In [32]:
# converting textual data to numerical data 

Vectorizer = TfidfVectorizer()

In [35]:
Vectorizer.fit(X)
X

array(['darrel lucushous dem aid didn’t even see comey’ letter jason chaffetz tweet',
       'daniel j flynnflynn hillari clinton big woman campu breitbart',
       'consortiumnews.comwhi truth might get fire', ...,
       'michael j de la merc rachel abramsmacy’ said receiv takeov approach hudson’ bay new york time',
       'alex ansarynato russia hold parallel exercis balkan',
       'david swansonwhat keep f-3 aliv'], dtype=object)

In [38]:
X = Vectorizer.transform(X)

In [40]:
print(X)

  (0, 27980)	0.2749650745266663
  (0, 24218)	0.24549606836407908
  (0, 16098)	0.41152381568827645
  (0, 15615)	0.27919531987024576
  (0, 13793)	0.2355845878757248
  (0, 8785)	0.22136452017328076
  (0, 7178)	0.28309646340484684
  (0, 6907)	0.2717513441333649
  (0, 6590)	0.3395843219966156
  (0, 5563)	0.23660110282600283
  (0, 4742)	0.3469045682183271
  (0, 958)	0.2582419179187741
  (1, 29698)	0.36799257932950036
  (1, 12236)	0.24880056097593603
  (1, 9588)	0.5242595805042315
  (1, 6535)	0.3173410216335356
  (1, 5315)	0.23272712583163743
  (1, 4375)	0.4606645580312347
  (1, 3768)	0.1859554590554668
  (1, 3087)	0.35520856940408263
  (2, 27870)	0.3657842933555564
  (2, 17500)	0.43210284748622374
  (2, 10321)	0.3068271309825366
  (2, 9375)	0.3450746133786227
  (2, 5815)	0.4046025649706647
  :	:
  (20797, 27391)	0.08222889881417556
  (20797, 26906)	0.31411839139790254
  (20797, 23553)	0.2486599147896777
  (20797, 22182)	0.27255928329508355
  (20797, 21888)	0.24962993538463268
  (20797, 18695

In [41]:
X

<20800x31215 sparse matrix of type '<class 'numpy.float64'>'
	with 203694 stored elements in Compressed Sparse Row format>

Train_test_split

In [46]:
X_train, X_test,Y_train , Y_test = train_test_split( X, Y, test_size = 0.2 , stratify = Y , random_state = 2)

In [47]:
# Training the model : logistic Regression 

model = LogisticRegression()

In [48]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

In [53]:
# accuracy score on training data 

X_train_predict= model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_predict, Y_train)



In [54]:
print("Accuracy score of training data :", training_data_accuracy)

Accuracy score of training data : 0.9715745192307692


In [55]:
# accuracy score on test data 

X_test_predict= model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_predict, Y_test)
print("Accuracy score of test data :", test_data_accuracy)




Accuracy score of test data : 0.9540865384615385


Making a Predictive system

In [59]:
X_new = X_test[1]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print("The news is real !")
else:
  print("Then news is fake !")


[0]
The news is real !


In [58]:
print(Y_test[1])

0
